# Canadian Household Income Analysis

# Setup

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import polars as pl
import pandas as pd
import numpy as np

Define some constants to make key column names easier to reference.

In [2]:
TOTAL_HOUSEHOLDS_COL = "HSBASHHD"
INSURANCE_COL = "HSEP001S"
PORTION_RET_INSUR_COL = "portion_retirement_insurance" # Our target column that we will calculate
INCOME_COL = "HSHNIAGG"

Read the data.

In [3]:
hs = pl.read_csv("./data/HouseholdSpend.csv")
ds = pl.read_csv("./data/DemoStats.csv")

# Check for initial null or NaN values
# Assert no NaNs in a DataFrame with float columns
assert not hs.drop_nans().shape[0] < hs.shape[0], "Spending DataFrame contains NaN values"
assert not ds.drop_nans().shape[0] < ds.shape[0], "Demographics DataFrame contains NaN values"
assert not hs.drop_nulls().shape[0] < hs.shape[0], "Spending DataFrame contains null values"
assert not ds.drop_nulls().shape[0] < ds.shape[0], "Demographics DataFrame contains null values"

display(hs.describe())
display(ds.describe())

statistic,CODE,GEO,HSBASHHD,HSHNIAGG,HSAGDISPIN,HSAGDISCIN,HSTT001,HSTE001,HSTX001,HSTC001,HSSH001S,HSFD001S,HSHO001S,HSHC001S,HSHF001S,HSTR001S,HSRE001S,HSPC001S,HSCL001S,HSED002S,HSRO001S,HSTA001S,HSGC001S,HSME001S,HSEP001S,HSMG001S,HSTE001ZBS,HSWH002S,HSWH028S,HSWH040S,HSWH041S,HSWH042S,HSSH001,HSSH002,HSSH003,HSSH004,…,HSTR002,HSTR003,HSTR004,HSTR005,HSTR006,HSTR007,HSTR008,HSTR009,HSTR058,HSTR010,HSTR011,HSTR012,HSTR014M,HSTR015,HSTR020,HSTR030,HSTR031,HSTR032,HSTR033,HSTR034,HSTR035,HSTR036,HSTR037,HSTR038,HSTR039,HSTR040,HSTR041,HSTR050,HSTR051,HSTR052,HSTR053,HSTR054,HSTR055,HSTR056,HSTR056A,HSTR056B,HSTR057
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""868970""","""868970""",868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,…,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,18.432629,2.2534e6,1.7170e6,1.2342e6,2.4008e6,2.1917e6,421335.834661,1.6065e6,388812.651163,278016.994627,111401.557243,109343.827301,76103.411113,271135.371662,87929.804561,45315.650319,70798.452452,34398.504565,3570.867859,70468.350013,25849.158365,33310.372386,115213.19111,48732.91062,209040.481816,67544.21004,4806.924846,112565.256385,15089.24985,9034.840695,388812.651163,303764.492909,89251.190609,88053.450315,…,236398.761036,118477.006209,114562.677761,14888.288571,8607.895903,91066.493288,188.957063,83.539013,105.41805,3725.371385,3469.552047,2092.59778,1376.954267,255.819338,1504.664209,116417.090618,3287.664753,10103.061471,12676.188385,18214.142317,100.530695,63415.559732,2639.560863,4612.181943,1368.200459,379.01064,989.189819,34736.610626,5995.306708,2258.469958,915.416319,23038.220756,71.1409,1322.434304,831.592595,490.84171,1135.621681
"""std""",null,null,69.379951,8.2787e6,6.3276e6,4.6340e6,9.1329e6,8.0562e6,1.5969e6,5.9255e6,1.3661e6,1.0471e6,420881.63654,429642.774397,290959.592849,1.0751e6,327376.60947,163377.943862,249311.269506,114999.916794,14386.745375,264786.580294,94955.831503,127397.296924,431645.982804,206454.726959,1.2713e6,341440.420828,40531.631801,898861.389623,51492.602769,35858.838458,1.3661e6,1.0270e6,292325.224526,287942.333453,…,990960.632583,507452.95803,494184.487741,58848.760548,45895.371562,398856.15598,794.022208,358.55285,497.256289,14112.764792,13213.369738,8173.498909,5705.767025,1422.161917,5673.303299,486744.921092,14400.182837,39378.141535,60281.31644,71889.568827,615.785308,284702.777168,13567.714202,14917.235034,5725.037991,2091.296014,3882.026395,101629.434794,19414.670337,7613.821326,5037.673958,71509.135272,257.018871,6638.799848,5076.885882,2097.649493,4203.801021
"""min""","""A0A0A0""","""FSALDU""",0.0,0.0,0.0,0.0,-24249.492537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.4599e7,0.0,0.0,-1.6806e7,-26676.98553,-204380.698719,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

statistic,CODE,GEO,ECYASQKM,ECYALSQKM,ECYBASPOP,ECYBASHHD,ECYBASHPOP,ECYBAS12P,ECYBAS15P,ECYBAS18P,ECYBAS19P,ECYBAS12HP,ECYBAS15HP,ECYBAS18HP,ECYBAS19HP,ECYBASTNGH,ECYBASADUH,ECYBASCF,ECYBASCFH,ECYBASKID,ECYBASLF,ECYPTAPOP,ECYPTA_0_4,ECYPTA_5_9,ECYPTA1014,ECYPTA1519,ECYPTA2024,ECYPTA2529,ECYPTA3034,ECYPTA3539,ECYPTA4044,ECYPTA4549,ECYPTA5054,ECYPTA5559,ECYPTA6064,ECYPTA6569,…,ECYRIMINDI,ECYRIMNEPA,ECYRIMPAKI,ECYRIMSRI,ECYRIMSASO,ECYRIMOCE,ECYRIMAUSS,ECYRIMOCEO,ECYPIMHPOP,ECYPIMNI,ECYPIMIM,ECYPIMP01,ECYPIM0110,ECYPIM1115,ECYPIM1621,ECYPIM22CY,ECYPIMNPER,ECYAIMHPOP,ECYAIMNI,ECYAIMIM,ECYAIM_0_5,ECYAIM_514,ECYAIM1524,ECYAIM2544,ECYAIM45P,ECYAIMNPER,ECYGENHPOP,ECYGEN1GEN,ECYGEN2GEN,ECYGEN3GEN,ECYTCAHPOP,ECYTCACIT,ECYTCA_U18,ECYTCA_18P,ECYNCANCIT,ECYNCA_U18,ECYNCA_18P
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""868970""","""868970""",868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,…,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,0.0,0.0,46.684199,18.432629,45.881762,41.254934,39.613594,37.819792,37.323999,40.336732,38.832472,37.263988,36.746572,3.072744,24.49923,12.76093,12.051364,12.740845,24.633679,46.684199,2.181318,2.390635,2.498651,2.672259,3.213171,3.308621,3.406466,3.258362,3.117668,2.858167,2.776588,2.859335,3.10898,2.80384,…,0.477122,0.005692,0.038764,0.008754,0.00028,0.007272,0.005106,0.002166,45.881762,31.717238,11.757474,4.721479,2.368195,1.391643,1.640656,1.635501,2.407051,45.881762,31.717238,11.757474,1.066154,2.036562,2.230265,5.299374,1.125119,2.407051,45.881762,14.168175,7.448384,24.265204,45.881762,40.528679,7.761493,32.767185,5.353084,0.85628,4.496803
"""std""",null,null,0.0,0.0,174.837877,69.379951,171.817976,152.539099,145.876662,139.320601,137.611402,149.038058,143.038376,137.181937,135.319262,12.468123,87.647782,50.126862,47.828225,49.02113,89.352097,174.837877,9.518254,10.400337,10.705186,10.30089,10.888878,10.894537,11.699239,11.371185,11.099312,10.485757,10.544898,11.657788,13.69638,12.729886,…,2.516032,0.123928,0.364373,0.142995,0.02439,0.118581,0.094428,0.057987,171.817976,149.721097,32.868441,14.28478,7.28958,4.605282,5.813031,5.761745,10.479555,171.817976,149.721097,32.868441,3.400699,6.112712,6.550715,14.905824,3.936661,10.479555,171.817976,40.185434,24.588753,131.818196,171.817976,163.866165,34.782061,130.449848,17.893635,3.211073,15.020402
"""min""","""A0A0A0""","""FSALDU""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",null,null,0.0,0.0,8.0,3.0,7.0,7.0,7.0,6.0,6.0,7.0,7.0,6.0,6.0,0.0,4.0,2.0,2.0,2.0,4.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,6.0,0.0,0.0,0.0,0.0,0.0,

Drop the zip code and geography level columns - these are just IDs that won't help our model.

In [4]:
hs.drop_in_place("CODE")
hs.drop_in_place("GEO")
ds.drop_in_place("GEO")
ds.drop_in_place("CODE")


CODE
str
"""A0A0A0"""
"""A0A1A0"""
"""A0A1B0"""
"""A0A1C0"""
"""A0A1E0"""
…
"""Y1A6W1"""
"""Y1A7A1"""
"""Y1A7A2"""


## Feature Engineering

Currently all the household spending variables represent total spending for entire neighbourhood; we will convert them to average per household spending which is probably a better predictor of per household insurance contribution. We don't want to confuse the model with some neighbourhoods having dramatically higher spending only because of high population.

Then we will construct our target variable, the portion of income a household spends on insurance and retirement savings.

In [5]:
total_households = hs.select(TOTAL_HOUSEHOLDS_COL)
hs.drop_in_place(TOTAL_HOUSEHOLDS_COL)
hs = hs.with_columns(
    pl.all() / total_households[TOTAL_HOUSEHOLDS_COL]
)

# Construct our target variable: portion of income spent on insurance and retirement
hs = hs.with_columns(
    (hs[INSURANCE_COL] / hs[INCOME_COL]).alias(PORTION_RET_INSUR_COL)
)

hs = hs.fill_nan(0) # replace NaNs resulting from zip codes with no households; we will remove these later
hs.describe()

statistic,HSHNIAGG,HSAGDISPIN,HSAGDISCIN,HSTT001,HSTE001,HSTX001,HSTC001,HSSH001S,HSFD001S,HSHO001S,HSHC001S,HSHF001S,HSTR001S,HSRE001S,HSPC001S,HSCL001S,HSED002S,HSRO001S,HSTA001S,HSGC001S,HSME001S,HSEP001S,HSMG001S,HSTE001ZBS,HSWH002S,HSWH028S,HSWH040S,HSWH041S,HSWH042S,HSSH001,HSSH002,HSSH003,HSSH004,HSSH053,HSSH054,HSSH005,…,HSTR003,HSTR004,HSTR005,HSTR006,HSTR007,HSTR008,HSTR009,HSTR058,HSTR010,HSTR011,HSTR012,HSTR014M,HSTR015,HSTR020,HSTR030,HSTR031,HSTR032,HSTR033,HSTR034,HSTR035,HSTR036,HSTR037,HSTR038,HSTR039,HSTR040,HSTR041,HSTR050,HSTR051,HSTR052,HSTR053,HSTR054,HSTR055,HSTR056,HSTR056A,HSTR056B,HSTR057,portion_retirement_insurance
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,…,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",113083.263311,86004.969701,61756.463377,124100.46217,110659.096698,21329.761528,81120.980552,19451.348892,14051.852189,5640.647583,5453.737856,3894.12655,13714.912167,4475.787785,2307.920882,3620.452159,1831.592429,180.070429,3552.323879,1258.671821,1687.535929,5767.651152,2440.703466,13441.365471,3679.226495,183.21153,8340.110651,767.014484,471.802312,19451.348892,15057.07041,3572.375804,3527.188921,3503.827985,23.360936,8.074081,…,5998.314594,5794.098686,754.827871,489.202334,4550.06848,10.131729,4.580577,5.551153,194.084179,180.690133,108.968722,71.721411,13.394046,71.132071,5989.931861,164.19669,523.370725,666.971454,967.513306,5.106435,3239.682473,127.909632,224.4644,70.716746,19.397331,51.319415,1655.533641,256.831429,101.69908,44.400143,1130.306389,3.304134,67.110823,42.685376,24.425447,51.881644,0.048015
"""std""",95888.112232,61585.741431,47271.395164,106423.582737,96165.001769,34249.337102,57520.845902,14437.227215,8735.079763,4150.74371,3486.151988,3706.807942,9177.026733,3721.066535,1627.426249,2964.140116,2626.886746,143.91476,2806.49432,896.55783,1738.505479,3501.73853,5563.278449,23340.657412,3268.849224,363.169609,20937.345568,875.526983,401.521054,14437.227215,10180.499762,3356.026359,3310.699569,3284.343463,36.142203,26.417176,…,4397.121656,4230.065806,621.170324,541.99211,3477.680417,12.036328,5.169206,9.316355,223.83379,214.586982,136.121851,94.865981,19.929029,83.535532,3689.98299,172.769453,370.270476,599.671703,676.483186,6.464252,2048.639749,158.369118,216.206197,66.25426,19.125852,55.045244,1714.792859,293.847957,113.030572,94.403067,1334.561812,2.892185,131.658002,108.022754,33.055399,51.994555,0.022317
"""min""",0.0,0.0,0.0,-24249.492537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-713372.555881,0.0,0.0,-821073.240312,-6402.348112,-21679.582212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",73127.635536,60115.360364,41531.8461

## Feature Selection

Let's remove some redundant features. We'll first drop features highly correlated with our target variable. Then, we'll read in the metadata file for the dataset, which is structured like a pre-order depth first search tree traversal. We'll use this metadata to identify and delete the non-"leaf node" features that don't give us any new info. For example, "Total Population" is redundant when we have "Total Male Population", "Total Female Population" and "Total Other Population". 

In [6]:
# drop columns used to calculate target or highly correlated with the target (related to income or insurance spend)
columns_to_drop = [INCOME_COL, "HSAGDISPIN", "HSAGDISCIN", INSURANCE_COL]
for col in columns_to_drop:
    hs.drop_in_place(col)


In [7]:
def get_columns_to_drop(metadata_file):
    # Read the metadata file
    df = pl.read_csv(metadata_file)
    
    #Initialize lists for variables to drop
    non_leaf_variables = []
    summary_variables = []
    
    # process rows in order to identify non-leaf nodes
    for i in range(len(df)):
        var = df['Variable'][i]
        hier_level = df['Hierarchy Level'][i]
        

        # Check if this is a summary variable (ends with AVG or MED)
        if var.endswith('AVG') or var.endswith('MED'):
            summary_variables.append(var)
            continue
        
        # Check if the next row exists and is a direct child
        if i < len(df) - 1 and df['Hierarchy Level'][i+1] == hier_level + 1:
            # This is a parent node (has at least one child)
            non_leaf_variables.append(var)
    
    #  Combine all var to drop
    columns_to_drop = list(set(non_leaf_variables +  summary_variables))
    
    return columns_to_drop


ds_columns_to_drop = get_columns_to_drop("data\DemoStats 2024 - Metadata.csv")
hs_columns_to_drop = get_columns_to_drop("data\HouseholdSpend 2024 - Metadata.csv")

# set aside insurance and retirement - this is our target variable

dropped_hs_columns = []
for col in hs_columns_to_drop:
    if col in hs.columns:
        hs.drop_in_place(col)
        dropped_hs_columns.append(col)
print(f"Dropped columns from hs: {', '.join(dropped_hs_columns)}")

dropped_ds_columns = []
for col in ds_columns_to_drop:
    if col in ds.columns:
        ds.drop_in_place(col)
        dropped_ds_columns.append(col)
print(f"Dropped columns from ds: {', '.join(dropped_ds_columns)}")


display(hs.describe())
display(ds.describe())

<>:31: SyntaxWarning: invalid escape sequence '\D'
<>:32: SyntaxWarning: invalid escape sequence '\H'
<>:31: SyntaxWarning: invalid escape sequence '\D'
<>:32: SyntaxWarning: invalid escape sequence '\H'
C:\Users\kevin\AppData\Local\Temp\ipykernel_10468\1563151703.py:31: SyntaxWarning: invalid escape sequence '\D'
  ds_columns_to_drop = get_columns_to_drop("data\DemoStats 2024 - Metadata.csv")
C:\Users\kevin\AppData\Local\Temp\ipykernel_10468\1563151703.py:32: SyntaxWarning: invalid escape sequence '\H'
  hs_columns_to_drop = get_columns_to_drop("data\HouseholdSpend 2024 - Metadata.csv")


Dropped columns from hs: HSHE011, HSTR050, HSCS011, HSSH010, HSFD990, HSSH030, HSHO001, HSCC001, HSSH031, HSHC001, HSTR039, HSCS001, HSHF005, HSHF002, HSSH041, HSHF001, HSSH040, HSTC001, HSSH050, HSTR002, HSTR008, HSHO014, HSHC022, HSHO010, HSSH003, HSSH016, HSSH047, HSSH002, HSTR004, HSHC005, HSHE001, HSSH037, HSTR011, HSHC002, HSCS003, HSTR010, HSTE001, HSTT001, HSWH042S, HSCC002, HSFD991, HSHC006, HSTR003, HSTE001ZBS, HSTR030, HSCC013, HSSH032, HSSH036, HSSH033, HSSH034, HSHE002, HSSH001, HSHC010, HSSH012, HSSH046, HSFD001, HSTR056, HSSH035, HSSH004, HSHO003, HSSH012B, HSHE010, HSHO018, HSHE020, HSTR001, HSHE012, HSHC004
Dropped columns from ds: ECYTCAHPOP, ECYTIMEU, ECYHNIHHD, ECYPNIHP15, ECYRIMWAF, ECYTIMSAF, ECYRIMHPOP, ECYEDAHPWK, ECYTIMEA, ECYRIMEEU, ECYPTAMED, ECYMOTSING, ECYMARNMCL, ECYRELCHR, ECYRIMAM, ECYRIMOCE, ECYPMAPOP, ECYRIMA, ECYHTY1PH, ECYHTYFHT, ECYPMAMED, ECYHTYHHD, ECYACTHPL, ECYVISHPOP, ECYTIMNAF, ECYEDUHP15, ECYMTNMED, ECYHFSC, ECYTIMSA, ECYPTAAVG, ECYHTY1FH, EC

statistic,HSTX001,HSSH001S,HSFD001S,HSHO001S,HSHC001S,HSHF001S,HSTR001S,HSRE001S,HSPC001S,HSCL001S,HSED002S,HSRO001S,HSTA001S,HSGC001S,HSME001S,HSMG001S,HSWH002S,HSWH028S,HSWH040S,HSWH041S,HSSH053,HSSH054,HSSH005,HSSH006,HSSH007,HSSH011,HSSH014,HSSH013,HSSH015,HSSH017,HSSH018,HSSH019,HSSH021,HSSH020,HSSH022,HSRM002A,…,HSHE011B,HSHE031,HSHE012M,HSHE032,HSHE013,HSHE015,HSHE016,HSHE021,HSHE023,HSTR005,HSTR006,HSTR007,HSTR009,HSTR058,HSTR012,HSTR014M,HSTR015,HSTR020,HSTR031,HSTR032,HSTR033,HSTR034,HSTR035,HSTR036,HSTR037,HSTR038,HSTR040,HSTR041,HSTR051,HSTR052,HSTR053,HSTR054,HSTR055,HSTR056A,HSTR056B,HSTR057,portion_retirement_insurance
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,…,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",21329.761528,19451.348892,14051.852189,5640.647583,5453.737856,3894.12655,13714.912167,4475.787785,2307.920882,3620.452159,1831.592429,180.070429,3552.323879,1258.671821,1687.535929,2440.703466,3679.226495,183.21153,8340.110651,767.014484,3503.827985,23.360936,8.074081,32.17807,4.934732,5734.197578,2039.657603,224.346864,450.89118,688.498413,42.610658,57.327097,22.868613,19.337479,154.819705,36.180773,…,50.669429,47.839276,48.257366,24.473367,196.022564,116.300456,314.245848,123.612696,218.320846,754.827871,489.202334,4550.06848,4.580577,5.551153,108.968722,71.721411,13.394046,71.132071,164.19669,523.370725,666.971454,967.513306,5.106435,3239.682473,127.909632,224.4644,19.397331,51.319415,256.831429,101.69908,44.400143,1130.306389,3.304134,42.685376,24.425447,51.881644,0.048015
"""std""",34249.337102,14437.227215,8735.079763,4150.74371,3486.151988,3706.807942,9177.026733,3721.066535,1627.426249,2964.140116,2626.886746,143.91476,2806.49432,896.55783,1738.505479,5563.278449,3268.849224,363.169609,20937.345568,875.526983,3284.343463,36.142203,26.417176,38.451252,11.039197,5211.756831,2124.710405,429.364967,417.349717,761.746528,40.965083,55.488358,29.495715,33.196496,323.674195,65.65381,…,58.162434,61.261459,55.079125,43.653086,179.932695,121.896706,467.674284,173.012319,308.065888,621.170324,541.99211,3477.680417,5.169206,9.316355,136.121851,94.865981,19.929029,83.535532,172.769453,370.270476,599.671703,676.483186,6.464252,2048.639749,158.369118,216.206197,19.125852,55.045244,293.847957,113.030572,94.403067,1334.561812,2.892185,108.022754,33.055399,51.994555,0.022317
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-821073.240312,-6402.348112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",8272.243558,12655.415474,10652.687589,4175.338127,4131.35436,2285.338647,9472.642127,2736.240782,1622.154486,2405.924724,669.953215,114.43201

statistic,ECYASQKM,ECYALSQKM,ECYBASPOP,ECYBASHHD,ECYBASHPOP,ECYBAS12P,ECYBAS15P,ECYBAS18P,ECYBAS19P,ECYBAS12HP,ECYBAS15HP,ECYBAS18HP,ECYBAS19HP,ECYBASTNGH,ECYBASADUH,ECYBASCF,ECYBASCFH,ECYBASKID,ECYBASLF,ECYPTA_0_4,ECYPTA_5_9,ECYPTA1014,ECYPTA1519,ECYPTA2024,ECYPTA2529,ECYPTA3034,ECYPTA3539,ECYPTA4044,ECYPTA4549,ECYPTA5054,ECYPTA5559,ECYPTA6064,ECYPTA6569,ECYPTA7074,ECYPTA7579,ECYPTA8084,…,ECYRIMCHIN,ECYRIMHONG,ECYRIMJAPA,ECYRIMSKOR,ECYRIMEAO,ECYRIMPHIL,ECYRIMVIET,ECYRIMSEAO,ECYRIMBANG,ECYRIMINDI,ECYRIMNEPA,ECYRIMPAKI,ECYRIMSRI,ECYRIMSASO,ECYRIMAUSS,ECYRIMOCEO,ECYPIMNI,ECYPIMP01,ECYPIM0110,ECYPIM1115,ECYPIM1621,ECYPIM22CY,ECYPIMNPER,ECYAIMNI,ECYAIM_0_5,ECYAIM_514,ECYAIM1524,ECYAIM2544,ECYAIM45P,ECYAIMNPER,ECYGEN1GEN,ECYGEN2GEN,ECYGEN3GEN,ECYTCA_U18,ECYTCA_18P,ECYNCA_U18,ECYNCA_18P
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,…,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",0.0,0.0,46.684199,18.432629,45.881762,41.254934,39.613594,37.819792,37.323999,40.336732,38.832472,37.263988,36.746572,3.072744,24.49923,12.76093,12.051364,12.740845,24.633679,2.181318,2.390635,2.498651,2.672259,3.213171,3.308621,3.406466,3.258362,3.117668,2.858167,2.776588,2.859335,3.10898,2.80384,2.278714,1.770631,1.112368,…,0.115093,0.022437,0.004211,0.020825,0.005081,0.090905,0.023022,0.010255,0.016249,0.477122,0.005692,0.038764,0.008754,0.00028,0.005106,0.002166,31.717238,4.721479,2.368195,1.391643,1.640656,1.635501,2.407051,31.717238,1.066154,2.036562,2.230265,5.299374,1.125119,2.407051,14.168175,7.448384,24.265204,7.761493,32.767185,0.85628,4.496803
"""std""",0.0,0.0,174.837877,69.379951,171.817976,152.539099,145.876662,139.320601,137.611402,149.038058,143.038376,137.181937,135.319262,12.468123,87.647782,50.126862,47.828225,49.02113,89.352097,9.518254,10.400337,10.705186,10.30089,10.888878,10.894537,11.699239,11.371185,11.099312,10.485757,10.544898,11.657788,13.69638,12.729886,10.420684,8.140557,5.13421,…,0.751078,0.268443,0.077976,0.225336,0.09944,0.712533,0.228892,0.125798,0.233412,2.516032,0.123928,0.364373,0.142995,0.02439,0.094428,0.057987,149.721097,14.28478,7.28958,4.605282,5.813031,5.761745,10.479555,149.721097,3.400699,6.112712,6.550715,14.905824,3.936661,10.479555,40.185434,24.588753,131.818196,34.782061,130.449848,3.211073,15.020402
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",0.0,0.0,8.0,3.0,7.0,7.0,7.0,6.0,6.0,7.0,7.0,6.0,6.0,0.0,4.0,2.0,2.0,2.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0

## Missing Values and Imputation

Let's join the two datasets together so we have a single feature matrix.

In [8]:
data = pl.concat([hs, ds], how="horizontal")
display(data.describe())

statistic,HSTX001,HSSH001S,HSFD001S,HSHO001S,HSHC001S,HSHF001S,HSTR001S,HSRE001S,HSPC001S,HSCL001S,HSED002S,HSRO001S,HSTA001S,HSGC001S,HSME001S,HSMG001S,HSWH002S,HSWH028S,HSWH040S,HSWH041S,HSSH053,HSSH054,HSSH005,HSSH006,HSSH007,HSSH011,HSSH014,HSSH013,HSSH015,HSSH017,HSSH018,HSSH019,HSSH021,HSSH020,HSSH022,HSRM002A,…,ECYRIMCHIN,ECYRIMHONG,ECYRIMJAPA,ECYRIMSKOR,ECYRIMEAO,ECYRIMPHIL,ECYRIMVIET,ECYRIMSEAO,ECYRIMBANG,ECYRIMINDI,ECYRIMNEPA,ECYRIMPAKI,ECYRIMSRI,ECYRIMSASO,ECYRIMAUSS,ECYRIMOCEO,ECYPIMNI,ECYPIMP01,ECYPIM0110,ECYPIM1115,ECYPIM1621,ECYPIM22CY,ECYPIMNPER,ECYAIMNI,ECYAIM_0_5,ECYAIM_514,ECYAIM1524,ECYAIM2544,ECYAIM45P,ECYAIMNPER,ECYGEN1GEN,ECYGEN2GEN,ECYGEN3GEN,ECYTCA_U18,ECYTCA_18P,ECYNCA_U18,ECYNCA_18P
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,…,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0,868970.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",21329.761528,19451.348892,14051.852189,5640.647583,5453.737856,3894.12655,13714.912167,4475.787785,2307.920882,3620.452159,1831.592429,180.070429,3552.323879,1258.671821,1687.535929,2440.703466,3679.226495,183.21153,8340.110651,767.014484,3503.827985,23.360936,8.074081,32.17807,4.934732,5734.197578,2039.657603,224.346864,450.89118,688.498413,42.610658,57.327097,22.868613,19.337479,154.819705,36.180773,…,0.115093,0.022437,0.004211,0.020825,0.005081,0.090905,0.023022,0.010255,0.016249,0.477122,0.005692,0.038764,0.008754,0.00028,0.005106,0.002166,31.717238,4.721479,2.368195,1.391643,1.640656,1.635501,2.407051,31.717238,1.066154,2.036562,2.230265,5.299374,1.125119,2.407051,14.168175,7.448384,24.265204,7.761493,32.767185,0.85628,4.496803
"""std""",34249.337102,14437.227215,8735.079763,4150.74371,3486.151988,3706.807942,9177.026733,3721.066535,1627.426249,2964.140116,2626.886746,143.91476,2806.49432,896.55783,1738.505479,5563.278449,3268.849224,363.169609,20937.345568,875.526983,3284.343463,36.142203,26.417176,38.451252,11.039197,5211.756831,2124.710405,429.364967,417.349717,761.746528,40.965083,55.488358,29.495715,33.196496,323.674195,65.65381,…,0.751078,0.268443,0.077976,0.225336,0.09944,0.712533,0.228892,0.125798,0.233412,2.516032,0.123928,0.364373,0.142995,0.02439,0.094428,0.057987,149.721097,14.28478,7.28958,4.605282,5.813031,5.761745,10.479555,149.721097,3.400699,6.112712,6.550715,14.905824,3.936661,10.479555,40.185434,24.588753,131.818196,34.782061,130.449848,3.211073,15.020402
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-821073.240312,-6402.348112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",8272.243558,12655.415474,10652.687589,4175.338127,4131.35436,2285.338647,9472.642127,2736.240782,1622.154486,2405.924724,669.953215,114.432011,2262.5102

Drop rows where total population is 0 or > 5% of the values are negative/invalid. These represent empty postal codes that will only add noise to our data.
Also drop columns where all values are zero or > 5% of the values are negative/invalid.

In [9]:
# Drop rows where total population is 0 or has too many invalid values
invalid_row_threshold = 0.05  # 5% threshold

# filter rows where > 5% of values are invalid
data = data.filter(
    ~(pl.sum_horizontal(pl.all().lt(0)) / pl.sum_horizontal(pl.all().is_not_null()) > invalid_row_threshold)
)

# filter out rows where total population is 0
data = data.filter(~pl.all_horizontal(pl.all().eq(0)))

# filter out columns where > 5% of values are invalid
numeric_cols = [s.name for s in data]
cols_to_drop = [col for col in numeric_cols if (data[col] < 0).sum() / hs.height > invalid_row_threshold]
data.drop(cols_to_drop)

# drop all zero columns
data.drop(
    [col for col in hs.columns if (data[col] == 0).all()]
)
display(data.describe())

statistic,HSTX001,HSSH001S,HSFD001S,HSHO001S,HSHC001S,HSHF001S,HSTR001S,HSRE001S,HSPC001S,HSCL001S,HSED002S,HSRO001S,HSTA001S,HSGC001S,HSME001S,HSMG001S,HSWH002S,HSWH028S,HSWH040S,HSWH041S,HSSH053,HSSH054,HSSH005,HSSH006,HSSH007,HSSH011,HSSH014,HSSH013,HSSH015,HSSH017,HSSH018,HSSH019,HSSH021,HSSH020,HSSH022,HSRM002A,…,ECYRIMCHIN,ECYRIMHONG,ECYRIMJAPA,ECYRIMSKOR,ECYRIMEAO,ECYRIMPHIL,ECYRIMVIET,ECYRIMSEAO,ECYRIMBANG,ECYRIMINDI,ECYRIMNEPA,ECYRIMPAKI,ECYRIMSRI,ECYRIMSASO,ECYRIMAUSS,ECYRIMOCEO,ECYPIMNI,ECYPIMP01,ECYPIM0110,ECYPIM1115,ECYPIM1621,ECYPIM22CY,ECYPIMNPER,ECYAIMNI,ECYAIM_0_5,ECYAIM_514,ECYAIM1524,ECYAIM2544,ECYAIM45P,ECYAIMNPER,ECYGEN1GEN,ECYGEN2GEN,ECYGEN3GEN,ECYTCA_U18,ECYTCA_18P,ECYNCA_U18,ECYNCA_18P
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,…,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",23716.777979,21628.151935,15624.396837,6271.893204,6064.066385,4329.918771,15249.749813,4976.673791,2566.200615,4025.617442,2036.566181,200.222135,3949.864917,1399.529952,1876.388299,2713.842916,4090.969221,203.714757,9273.453544,852.851177,3895.941841,25.975261,8.977653,35.779122,5.486978,6375.912392,2267.915957,249.453551,501.350471,765.548362,47.379222,63.74258,25.427842,21.501539,172.145599,40.229768,…,0.127973,0.024948,0.004682,0.023155,0.005649,0.101079,0.025598,0.011402,0.018068,0.530517,0.006329,0.043102,0.009734,0.000311,0.005677,0.002408,35.266718,5.249861,2.633219,1.547382,1.824262,1.81853,2.676424,35.266718,1.185468,2.264474,2.479854,5.892428,1.251032,2.676424,15.753737,8.281933,26.980726,8.630083,36.434165,0.952107,5.000041
"""std""",35322.476515,13589.668736,7763.405219,3898.41362,3132.451504,3659.395109,8380.73619,3592.093264,1510.666713,2852.777939,2693.572255,137.820392,2680.974863,834.6471,1733.86223,5802.795805,3193.234969,377.459188,21880.939129,882.681155,3235.185242,37.209337,27.710174,38.924482,11.509627,5109.857075,2121.775199,445.782966,410.335941,765.642896,40.497009,54.905236,30.038027,34.333695,336.907528,68.043464,…,0.790949,0.282955,0.08221,0.237497,0.104841,0.750662,0.241223,0.132601,0.24606,2.647741,0.130664,0.383978,0.150753,0.025719,0.099556,0.061141,157.479696,14.970532,7.641126,4.831262,6.102293,6.048143,11.017723,157.479696,3.566164,6.405522,6.862595,15.606206,4.13208,11.017723,42.078629,25.794672,138.734736,36.574361,137.069161,3.372485,15.758954
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-821073.240312,-6402.348112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",10498.595672,14442.835295,11815.362034,4599.260584,4582.547079,2678.14736,10816.930251,3188.144857,1818.227831,2719.19476,846.466374,130.370373,261

Some of the spending variables have negative values which is obviously not valid. We will impute them with the median. We can see from the display() output that some columns' max spending values are in the 9-figure range. This is way outside a typical neighbourhood's spend in any category. We will cap extreme outliers by truncating all values above 3 * IQR for all the columns in the spending dataframe.

Finally, we will standardize the data using z-scores.

In [ ]:
class NegativeValueImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.medians = None
        
    def fit(self, X, y=None):
        # X can be a DataFrame or np  array
        if isinstance(X, pd.DataFrame):
            self.medians = {}
            for col in X.columns:
                # Calculate median of non-negative values
                non_negative_values = X.loc[X[col] >= 0, col]
                self.medians[col] = non_negative_values.median() if len(non_negative_values) > 0 else 0
        else:
            self.medians = []
            for i in range(X.shape[1]):
                non_negative_values = X[:, i][X[:, i] >= 0]
                self.medians.append(np.median(non_negative_values) if len(non_negative_values) > 0 else 0)
                
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        if isinstance(X_copy, pd.DataFrame):
            for col in X_copy.columns:
                mask = X_copy[col] < 0
                X_copy.loc[mask, col] = self.medians[col]
        else:  # numpy array
            for i in range(X_copy.shape[1]):
                mask = X_copy[:, i] < 0
                X_copy[mask, i] = self.medians[i]
                

        return X_copy

class IQRClippingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, multiplier= 3):
        self.multiplier = multiplier
        self.upper_bounds_ = {  }   


    def fit(self, X, y=None):
        df = pl.DataFrame(X)
          
        for col in df.columns:
            q1 = df[col].quantile(.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            self.upper_bounds_[col] = q3 + self.multiplier * iqr
        
        return self
    
    def transform(self, X):
        df = pl.DataFrame(X)    
         
        for col in df.columns:  
            if col in self.upper_bounds_:
                upper_bound = self.upper_bounds_[ col]
                df = df.with_columns(
                     pl.when(pl.col(col) > upper_bound)
                    .then(upper_bound)
                    .otherwise(pl.col(col)) 
                    .alias(col) )
        
        return  df.to_numpy()

Before fitting NegativeValueImputer:
shape: (9, 758)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆ HSTX001   ┆ HSSH001S  ┆ HSFD001S  ┆ … ┆ ECYTCA_U1 ┆ ECYTCA_18 ┆ ECYNCA_U1 ┆ ECYNCA_1 │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ 8         ┆ P         ┆ 8         ┆ 8P       │
│ str       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 781511.0  ┆ 781511.0  ┆ 781511.0  ┆ … ┆ 781511.0  ┆ 781511.0  ┆ 781511.0  ┆ 781511.0 │
│ null_coun ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 0.0      │
│ t         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ mean      ┆ 23716.777 ┆ 21628.151 ┆ 

AssertionError: StandardScaler failed: Standard deviation is not approximately 1.

In [11]:
y = data[PORTION_RET_INSUR_COL]
X = data.drop(PORTION_RET_INSUR_COL)

# Impute negative values and clip outliers
# Construct the pipeline
preprocess = Pipeline([
    ('impute_negatives', NegativeValueImputer()),
    ('clip_outliers', IQRClippingTransformer(multiplier=3)),
    ('standardize', StandardScaler())
])

# Apply the pipeline to the data
X_processed = preprocess.fit_transform(X.to_pandas())

# Convert back to Polars DataFrame
X_processed = pl.DataFrame(X_processed)

# set columns
X_processed.columns = X.columns

display(X_processed.describe())

statistic,HSTX001,HSSH001S,HSFD001S,HSHO001S,HSHC001S,HSHF001S,HSTR001S,HSRE001S,HSPC001S,HSCL001S,HSED002S,HSRO001S,HSTA001S,HSGC001S,HSME001S,HSMG001S,HSWH002S,HSWH028S,HSWH040S,HSWH041S,HSSH053,HSSH054,HSSH005,HSSH006,HSSH007,HSSH011,HSSH014,HSSH013,HSSH015,HSSH017,HSSH018,HSSH019,HSSH021,HSSH020,HSSH022,HSRM002A,…,ECYRIMCHIN,ECYRIMHONG,ECYRIMJAPA,ECYRIMSKOR,ECYRIMEAO,ECYRIMPHIL,ECYRIMVIET,ECYRIMSEAO,ECYRIMBANG,ECYRIMINDI,ECYRIMNEPA,ECYRIMPAKI,ECYRIMSRI,ECYRIMSASO,ECYRIMAUSS,ECYRIMOCEO,ECYPIMNI,ECYPIMP01,ECYPIM0110,ECYPIM1115,ECYPIM1621,ECYPIM22CY,ECYPIMNPER,ECYAIMNI,ECYAIM_0_5,ECYAIM_514,ECYAIM1524,ECYAIM2544,ECYAIM45P,ECYAIMNPER,ECYGEN1GEN,ECYGEN2GEN,ECYGEN3GEN,ECYTCA_U18,ECYTCA_18P,ECYNCA_U18,ECYNCA_18P
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,…,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0,781511.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",-1.8349e-16,-6.2607e-16,7.8823e-16,-8.8348e-16,-1.3151e-15,-1.0947e-17,-3.6429e-16,-2.1915e-16,6.5338e-16,7.7154e-16,6.0534e-17,7.4539e-16,-3.9790e-16,-1.0128e-16,-6.9953e-16,-1.2278e-16,-1.0738e-16,-5.4481e-16,1.9517e-16,-1.3947e-16,6.4734e-17,-1.1767e-16,-8.4600e-17,1.6702e-16,-1.1843e-16,8.8642e-16,5.1257e-16,4.8914e-17,-2.2826e-16,-4.0484e-16,-4.0870e-16,-1.6740e-16,1.3078e-16,-9.6229e-17,-3.6795e-17,1.6911e-17,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.2226e-17,3.4904e-17,-6.8671e-17,1.2478e-16,4.4987e-17,1.5009e-16,4.1814e-17,7.2226e-17,1.4241e-16,-3.3013e-17,-8.9646e-18,-3.4377e-17,-1.7691e-16,4.1814e-17,-6.1207e-17,-3.3549e-18,7.0008e-18,2.5112e-17,-8.4591e-17,4.0550e-18,-4.1250e-17
"""std""",1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001,1.000001
"""min""",-1.316828,-2.168847,-2.867458,-2.675862,-2.818175,-1.910454,-2.458588,-2.002602,-2.463781,-2.203985,-1.241381,-2.05306,-2.10579,-1.872657,-1.978248,-1.305182,-1.371866,-1.306016,-1.366635,-1.695644,-1.239219,-0.747991,-0.769673,-0.942896,-0.791453,-1.400408,-1.41785,-0.860757,-1.65337,-1.306939,-1.325703,-1.32756,-1.014225,-0.930242,-0.810018,-1.055725,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.000403,-0.699706,-0.641108,-0.602974,-0.658345,-0.690702,-0.613178,-1.000403,-0.640055,-0.682589,-0.627615,-0.66943,-0.567226,-0.613178,-0.714771,-0.781651,-0.979982,-0.921649,-0.98155,-0.535991,-0.677304
"""25%""",-0.675451,-0.692639,-0.6633